# **Programación Lineal Entera-Mixta (MILP)**

**Ejercicio 1**: Considera el problema de operación de un mercado eléctrico con el que hemos venido trabajando en las prácticas anteriores. Recordemos que ste mercado está compuesto por $N_{G}$ productores de electricidad que han de satisfacer una demanda $D$; en nuestro caso particular, $N_{G} = 3$.

En esta ocasión, sin embargo, vamos a suponer que, como realmente ocurre en la práctica, si bien el coste de producción de cada productor se puede aproximar mediante una *función lineal por tramos o a trozos*, esta **función no es convexa**. Vimos un ejemplo de este tipo de función en el último apartado de la Práctica 2, en concreto,

<center>

|  Productor    | Tramo 1   | Tramo 2  | Tramo 3 |
| ------------  | --------     | ---      | ---    |  
| 1            | (50, 10)      | (20, 15) | (30, 45)|
| 2            | (10, 5)       | (50, 26) | (60, 15)|   
| 3            | (100, 25)     | (50, 5) | (50, 30)|   

</center>

donde cada par $(l_{i,j}, m_{i,j})$ con $i, j = 1, \ldots, 3$ en la tabla especifica la longitud o anchura del trozo lineal $l_{i,j}$ junto con su pendiente $m_{i,j}$.



**Ejercicio 1.1**: Construye un modelo de programación matemática para liquidar este mercado, esto es, para decidir cuánto debe producir cada generador para una demanda $D = 200$. Recuerda que el modelado de las funciones de coste lineales por tramos **no convexas** debe ser tal que se asegure que cada tramo se rellena *de forma secuencial*.

In [1]:
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.0 MB/s eta 0:00:00


In [2]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Mercado_MILP = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [3]:
Mercado_MILP.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set Prod;                       # Conjunto de productores
set Tram ordered;               # Conjunto ordenado de tramos de las funciones de coste
param D >= 0;                   # Demanda del mercado
param lon {Prod, Tram} >= 0;    # Longitud de cada tramo
param slope {Prod, Tram} >= 0;  # Pendiente de cada tramo

var u {Prod, Tram} binary;      # Variables binarias para garantizar que los
                                # tramos se rellenarán de forma secuencial
var p {i in Prod, j in Tram} >= 0;  # Producción por generador


minimize coste_total:
    sum{i in Prod, j in Tram} slope[i,j]*p[i,j];
s.t. balance: sum{i in Prod, j in Tram} p[i,j] = D;
     limit_tramo_l{i in Prod, j in Tram: ord(j) > 1}: p[i,j-1] >= lon[i,j-1] * u[i,j];
     limit_tramo_u{i in Prod, j in Tram}: p[i,j] <= lon[i,j] * u[i,j];
     seq_fill{i in Prod, j in Tram: ord(j) > 1}: u[i,j] <= u[i,j-1];
""")

Obsérvese en el código anterior que la restricción "seq_fill" (que garantiza el
rellenado secuencial de los tramos de las funciones de coste) no se aplica al primer tramo (esto es, al tramo con ord(j) = 1). En caso contrario, el código produciría un error al intentar acceder a un índice "j-1" inexistente.

Nótese también que hemos hecho explícito que "Tram" es un conjunto ordenado escribiendo "set Tram ordered;".

In [4]:
def preparar_datos_mercado():
    import pandas as pd
    import numpy as np

    # Conjunto de tramos o piezas:
    tramos = [1, 2, 3]  # Asignamos a los tramos números naturales para los que
                        # AMPL preserva su orden y permite trabajar con "ord()"
                        # y utilizar expresiones tipo "j-1".

    # Conjunto de productores:
    productores = ["G1", "G2", "G3"]

    # Demanda del mercado:
    demanda = 200

    # Creamos un data frame que contenga la longitud de los tramos
    lon_df = pd.DataFrame(
        np.array(
            [
                [50, 20, 30],
                [10, 50, 60],
                [100, 50, 50],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    # Creamos un data frame que contenga las pendientes de los tramos
    slope_df = pd.DataFrame(
        np.array(
            [
                [10, 15, 45],
                [5, 26, 15],
                [25, 5, 30],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    return demanda, tramos, productores, lon_df, slope_df

In [5]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
demanda, tramos, productores, lon_df, slope_df = preparar_datos_mercado()

Mercado_MILP.set["Prod"] = productores
Mercado_MILP.set["Tram"] = tramos
Mercado_MILP.get_parameter("lon").set_values(lon_df)
Mercado_MILP.get_parameter("slope").set_values(slope_df)
Mercado_MILP.get_parameter("D").set(demanda)

In [6]:
Mercado_MILP.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Mercado_MILP.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 3200
7 simplex iterations


In [7]:
p = Mercado_MILP.get_variable("p") # Producción por generador y tramo
u = Mercado_MILP.get_variable("u") # Variables binarias para garantizar el rellenado secuencial
df_p = p.get_values().to_pandas()
df_u = u.get_values().to_pandas()
coste_total = Mercado_MILP.get_objective("coste_total").value()
precio = Mercado_MILP.get_constraint("balance").dual()
df_p = df_p.round(3)
print(df_p)
print(f"Coste total: {coste_total:.3f}")
print(f"Precio del mercado: {precio:.1f}")

print(df_u)

# Resumimos lo que produce cada productor:
df_Pg = df_p.groupby(level='index0').sum()
print(df_Pg)

               p.val
index0 index1       
G1     1          40
       2           0
       3           0
G2     1          10
       2           0
       3           0
G3     1         100
       2          50
       3           0
Coste total: 3200.000
Precio del mercado: 0.0
               u.val
index0 index1       
G1     1           1
       2           0
       3           0
G2     1           1
       2           1
       3           0
G3     1           1
       2           1
       3           0
        p.val
index0       
G1         40
G2         10
G3        150


En un problema MILP, las variables duales carecen de sentido. De hecho, en el problema práctico que nos ocupa, es harto interesante plantearse la siguiente pregunta: **¿Se puede encontrar un precio de mercado que induzca a los tres productores (que tratan de maximizar su beneficio como agentes "egoístas" que son) a estar satisfechos con las cantidades óptimas de producción dictadas por el mercado?**

Por otra parte, en la práctica, las funciones de coste de los productores suelen exhibir más rasgos "no convexos". Por ejemplo, es muy frecuente que estas funciones presenten lo que se conoce como "zonas muertas o prohibidas". Matemáticamente, una zona muerta no es más que la presencia de un "hueco" en el dominio de definición de la función de costes. Más concretamente, suele existir una producción mínima $\underline{P}$ tal que el productor no puede producir en el intervalo abierto $(0, \underline{P})$. Asimismo, producir a producción mínima suele acarrear un coste *fijo* $c$ que, para $P \geq \underline{P}$, se añade al coste *variable* lineal a trozos con el que hemos venido trabajando hasta ahora.

**Ejercicio 1.2**: Extiende el modelo MILP de funcionamiento de mercado para tener en cuenta que los tres productores exhiben una zona prohibida de producción $(0, \underline{P}_{i})$, $i =1, 2, 3$, con un coste fijo a producción mínima $c_{i}$. Resuelve este modelo para los valores numéricos indicados en la siguiente tabla, y discute la solución (con respecto a la del caso anterior):

<center>

|  Productor    | $\underline{P}_{i}$ | $c_i$  |
| ------------  | --------     | ---      |   
| 1            | 50            | 100       |
| 2            | 50            | 500      |    
| 3            | 0             | 0        |    

</center>

In [8]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Mercado_MILP2 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [9]:
Mercado_MILP2.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set Prod;                       # Conjunto de productores
set Tram ordered;               # Conjunto ordenado de tramos de las funciones de coste
param D >= 0;                   # Demanda del mercado
param lon {Prod, Tram} >= 0;    # Longitud de cada tramo
param slope {Prod, Tram} >= 0;  # Pendiente de cada tramo
param Pmin {Prod} >= 0;         # Producción mínima de cada productor
param c {Prod} >= 0;            # Coste fijo a producción mínima

var u {Prod, Tram} binary;      # Variables binarias para garantizar que los
                                # tramos se rellenarán de forma secuencial
var p {i in Prod, j in Tram} >= 0;  # Producción por generador
var Pg {Prod} >= 0;             # Producción total por productor


minimize coste_total:
    sum{i in Prod} (u[i,1] * c[i]) + sum{i in Prod, j in Tram} (slope[i,j]*p[i,j]);
s.t. balance: sum{i in Prod} (u[i,1] * Pmin[i]) + sum{i in Prod, j in Tram} p[i,j] = D;
     limit_tramo_l{i in Prod, j in Tram: ord(j) > 1}: p[i,j-1] >= lon[i,j-1] * u[i,j];
     limit_tramo_u{i in Prod, j in Tram}: p[i,j] <= lon[i,j] * u[i,j];
     seq_fill{i in Prod, j in Tram: ord(j) > 1}: u[i,j] <= u[i,j-1];
     prod_total{i in Prod}: Pg[i] = Pmin[i]*u[i,1] + sum{j in Tram} p[i,j];
""")

In [10]:
def preparar_datos_mercado2():
    import pandas as pd
    import numpy as np

    # Conjunto de tramos o piezas:
    tramos = [1, 2, 3]  # Asignamos a los tramos números naturales para los que
                        # AMPL preserva su orden y permite trabajar con "ord()"
                        # y utilizar expresiones tipo "j-1".

    # Conjunto de productores:
    productores = ["G1", "G2", "G3"]

    # Demanda del mercado:
    demanda = 200

    # Creamos un data frame que contenga la longitud de los tramos
    lon_df = pd.DataFrame(
        np.array(
            [
                [50, 20, 30],
                [10, 50, 60],
                [100, 50, 50],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    # Creamos un data frame que contenga las pendientes de los tramos
    slope_df = pd.DataFrame(
        np.array(
            [
                [10, 15, 45],
                [5, 26, 15],
                [25, 5, 30],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    # Datos de potencia mínima:

    Pmin =[50, 50, 0]

    # Coste fijo:

    c = [100, 500, 0]


    return demanda, tramos, productores, lon_df, slope_df, Pmin, c

In [11]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
demanda, tramos, productores, lon_df, slope_df, Pmin, c = preparar_datos_mercado2()

Mercado_MILP2.set["Prod"] = productores
Mercado_MILP2.set["Tram"] = tramos
Mercado_MILP2.get_parameter("lon").set_values(lon_df)
Mercado_MILP2.get_parameter("slope").set_values(slope_df)
Mercado_MILP2.get_parameter("D").set(demanda)
Mercado_MILP2.param["Pmin"] = {productores[i]: j for i, j in enumerate(Pmin)}
Mercado_MILP2.param["c"] = {productores[i]: j for i, j in enumerate(c)}

In [12]:
Mercado_MILP2.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Mercado_MILP2.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 1950
8 simplex iterations


In [13]:
p = Mercado_MILP2.get_variable("p") # Producción por generador y tramo
u = Mercado_MILP2.get_variable("u") # Variables binarias para garantizar el rellenado secuencial
df_p = p.get_values().to_pandas()
df_u = u.get_values().to_pandas()
coste_total = Mercado_MILP2.get_objective("coste_total").value()
precio = Mercado_MILP2.get_constraint("balance").dual()
df_p = df_p.round(3)
print(df_p)
print(f"Coste total: {coste_total:.3f}")
print(f"Precio del mercado: {precio:.1f}")

print(df_u)

# Resumimos lo que produce cada productor:
Pg = Mercado_MILP2.get_variable("Pg")
df_Pg = Pg.get_values().to_pandas()
print(df_Pg)



               p.val
index0 index1       
G1     1          50
       2          20
       3           0
G2     1          10
       2           0
       3           0
G3     1          20
       2           0
       3           0
Coste total: 1950.000
Precio del mercado: 0.0
               u.val
index0 index1       
G1     1           1
       2           1
       3           0
G2     1           1
       2           0
       3           0
G3     1           1
       2           0
       3           0
    Pg.val
G1     120
G2      60
G3      20


**Ejercicio 2** (Problema de la mochila): Considera un armador que tiene un carguero con capacidad de hasta 1500 toneladas. El carguero transporta contenedores de diferentes pesos para una determinada ruta. En la ruta actual el carguero puede transportar algunos de los siguientes contenedores:

<center>

| Contenedor | c1  | c2  | c3  | c4  | c5  | c6  | c7  | c8  | c9  | c10 |
|-------------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| **Peso**    | 100 | 155 | 50  | 112 | 70  | 80  | 60  | 118 | 110 | 55  |
| **Valor**   | 50   | 65   | 30| 60   | 35   | 37 | 29   | 48   | 53   | 100   |
| **Unidades**| 3   | 4   | 1   | 2   | 1   | 1   | 5   | 8   | 3   | 1   |

</center>

La última fila de la tabla indica el número existente de contenedores con un cargamento del valor y peso indicados.

**Ejercicio 2.1**: Formula un modelo de Programación Matemática para determinar el envío (conjunto de contenedores) que maximiza el valor de la carga transportada por el carguero a fletar.

$$
\begin{array}{rrcl}
    \min & \sum_{c \in C} v_c x_{c}&\\
    {\rm sujeto\  a}&  \sum_{c \in C} p_c x_{c} & \leq& \overline{P}\\
    & x_c  &\leq& N_c, \ \forall c \in C\\
    &x_c& \in & \mathbb{Z}_{\geq 0}, \ \forall c \in C
\end{array}
$$

In [14]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Carguero = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [15]:
Carguero.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont} integer <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
""")


In [16]:
# Introducimos el valor, el peso y el número de unidades de los contenedores
# como diccionarios
valor = {
    "c1": 50,
    "c2": 65,
    "c3": 30,
    "c4": 60,
    "c5": 35,
    "c6": 37,
    "c7": 29,
    "c8": 48,
    "c9": 53,
    "c10": 100,
}

peso = {
    "c1": 100,
    "c2": 155,
    "c3": 50,
    "c4": 112,
    "c5": 70,
    "c6": 80,
    "c7": 60,
    "c8": 118,
    "c9": 110,
    "c10": 55,
}

n = {
    "c1": 3,
    "c2": 4,
    "c3": 1,
    "c4": 2,
    "c5": 1,
    "c6": 1,
    "c7": 5,
    "c8": 8,
    "c9": 3,
    "c10": 1,
}

Carguero.set["cont"] = list(valor.keys())
Carguero.param["valor"] = {c: v for c, v in valor.items()}
Carguero.param["peso"] = {c: p for c, p in peso.items()}
Carguero.param["n"] = {c: num for c, num in n.items()}
Carguero.param["cap"] = 1500


In [17]:
Carguero.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Carguero.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 806
6 simplex iterations


In [18]:
x = Carguero.get_variable("x") # Número de cargueros de cada tipo
df_x = x.get_values().to_pandas()
print(df_x)
valor_total = Carguero.get_objective("valor_total").value()
print(f"Valor total: {valor_total:.3f}")




     x.val
c1       3
c10      1
c2       1
c3       1
c4       2
c5       0
c6       1
c7       5
c8       0
c9       3
Valor total: 806.000


**Ejercicio 2.2**: Resuelve el problema LP relajado y discute los inconvenientes que plantearía redondear la solución obtenida.

In [19]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroR = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [20]:
CargueroR.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
""")


In [21]:
CargueroR.set["cont"] = list(valor.keys())
CargueroR.param["valor"] = {c: v for c, v in valor.items()}
CargueroR.param["peso"] = {c: p for c, p in peso.items()}
CargueroR.param["n"] = {c: num for c, num in n.items()}
CargueroR.param["cap"] = 1500

In [22]:
CargueroR.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroR.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 814.1612903
1 simplex iterations


In [23]:
xR = CargueroR.get_variable("x") # Número de cargueros de cada tipo
df_xR = xR.get_values().to_pandas()
print(df_xR)
valor_total_R = CargueroR.get_objective("valor_total").value()
print(f"Valor total: {valor_total_R:.3f}")




        x.val
c1   3.000000
c10  1.000000
c2   0.587097
c3   1.000000
c4   2.000000
c5   1.000000
c6   1.000000
c7   5.000000
c8   0.000000
c9   3.000000
Valor total: 814.161


**Ejercicio 3**: Considera una versión reducida del problema del carguero, en que solo se tienen disponibles los tres primeros contenedores que se indican en la tabla y la capacidad del carguero es de 400.

Con la ayuda de un solver para resolver LPs, aplica ramificación y cota para obtener la solución óptima del problema a partir de su relajación lineal.

Resolvemos la relajación lineal de la versión reducida del problema.

In [69]:
# Introducimos el valor, el peso y el número de unidades de los contenedores
# como diccionarios
valor = {
    "c1": 50,
    "c2": 65,
    "c3": 30,
}

peso = {
    "c1": 100,
    "c2": 155,
    "c3": 50,
}

n = {
    "c1": 3,
    "c2": 4,
    "c3": 1,
}

In [71]:
CargueroR.set["cont"] = list(valor.keys())
CargueroR.param["valor"] = {c: v for c, v in valor.items()}
CargueroR.param["peso"] = {c: p for c, p in peso.items()}
CargueroR.param["n"] = {c: num for c, num in n.items()}
CargueroR.param["cap"] = 400

In [72]:
CargueroR.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroR.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 200.9677419
0 simplex iterations


In [73]:
xR = CargueroR.get_variable("x") # Número de cargueros de cada tipo
df_xR = xR.get_values().to_pandas()
print(df_xR)
valor_total_R = CargueroR.get_objective("valor_total").value()
print(f"Valor total: {valor_total_R:.3f}")




       x.val
c1  3.000000
c2  0.322581
c3  1.000000
Valor total: 200.968


Ramificamos la variable $x_{c_2}$: Resolvemos **$P_1 \ (x_{c_2} \geq 1$)** y **$P_2 \ (x_{c_2} \leq 0$)**



In [74]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP1 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [75]:
CargueroP1.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding: x["c2"] >= 1;
""")


In [76]:
CargueroP1.set["cont"] = list(valor.keys())
CargueroP1.param["valor"] = {c: v for c, v in valor.items()}
CargueroP1.param["peso"] = {c: p for c, p in peso.items()}
CargueroP1.param["n"] = {c: num for c, num in n.items()}
CargueroP1.param["cap"] = 400

In [77]:
CargueroP1.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP1.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 192.5
0 simplex iterations


In [78]:
xP1 = CargueroP1.get_variable("x") # Número de cargueros de cada tipo
df_xP1 = xP1.get_values().to_pandas()
print(df_xP1)
valor_total_P1 = CargueroP1.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P1:.3f}")




    x.val
c1   1.95
c2   1.00
c3   1.00
Valor total: 192.500


In [79]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP2 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [80]:
CargueroP2.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding: x["c2"] <= 0;
""")


In [81]:
CargueroP2.set["cont"] = list(valor.keys())
CargueroP2.param["valor"] = {c: v for c, v in valor.items()}
CargueroP2.param["peso"] = {c: p for c, p in peso.items()}
CargueroP2.param["n"] = {c: num for c, num in n.items()}
CargueroP2.param["cap"] = 400

In [82]:
CargueroP2.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP2.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 180
0 simplex iterations


In [83]:
xP2 = CargueroP2.get_variable("x") # Número de cargueros de cada tipo
df_xP2 = xP2.get_values().to_pandas()
print(df_xP2)
valor_total_P2 = CargueroP2.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P2:.3f}")




    x.val
c1      3
c2      0
c3      1
Valor total: 180.000


El nodo $P_2$ queda pues agotado y nos permite actualizar la cota inferior del valor objetivo óptimo del problema original a 180. Consecuentemente, podemos actualizar también la cota superior a 192 gracias a $P_1$.

Ramificamos $P_1$ y resolvemos **$P_3 \ (x_{c_1} \geq 2$)** y **$P_4 \ (x_{c_1} \leq 1$)**

In [84]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP3 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [85]:
CargueroP3.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 1;
    bounding2: x["c1"] >= 2;
""")


In [86]:
CargueroP3.set["cont"] = list(valor.keys())
CargueroP3.param["valor"] = {c: v for c, v in valor.items()}
CargueroP3.param["peso"] = {c: p for c, p in peso.items()}
CargueroP3.param["n"] = {c: num for c, num in n.items()}
CargueroP3.param["cap"] = 400

In [87]:
CargueroP3.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP3.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 192
0 simplex iterations


In [88]:
xP3 = CargueroP3.get_variable("x") # Número de cargueros de cada tipo
df_xP3 = xP3.get_values().to_pandas()
print(df_xP3)
valor_total_P3 = CargueroP3.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P3:.3f}")




    x.val
c1    2.0
c2    1.0
c3    0.9
Valor total: 192.000


In [41]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP4 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [89]:
CargueroP4.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 1;
    bounding2: x["c1"] <= 1;
""")


In [90]:
CargueroP4.set["cont"] = list(valor.keys())
CargueroP4.param["valor"] = {c: v for c, v in valor.items()}
CargueroP4.param["peso"] = {c: p for c, p in peso.items()}
CargueroP4.param["n"] = {c: num for c, num in n.items()}
CargueroP4.param["cap"] = 400

In [91]:
CargueroP4.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP4.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 184.8387097
0 simplex iterations


In [92]:
xP4 = CargueroP4.get_variable("x") # Número de cargueros de cada tipo
df_xP4 = xP4.get_values().to_pandas()
print(df_xP4)
valor_total_P4 = CargueroP4.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P4:.3f}")




       x.val
c1  1.000000
c2  1.612903
c3  1.000000
Valor total: 184.839


Ramificamos sobre $P_3$ y resolvemos **$P_5 \ (x_{c_3} \geq 1$)** y **$P_6 \ (x_{c_3} \leq 0$)**

In [93]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP5 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [95]:
CargueroP5.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 1;
    bounding2: x["c1"] >= 2;
    bounding3: x["c3"] >= 1;

""")


In [96]:
CargueroP5.set["cont"] = list(valor.keys())
CargueroP5.param["valor"] = {c: v for c, v in valor.items()}
CargueroP5.param["peso"] = {c: p for c, p in peso.items()}
CargueroP5.param["n"] = {c: num for c, num in n.items()}
CargueroP5.param["cap"] = 400

In [98]:
CargueroP5.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: infeasible problem
0 simplex iterations

suffix dunbdd OUT;


$P_5$ es infactible y esa vía queda agotada.

In [52]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP6 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [99]:
CargueroP6.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 1;
    bounding2: x["c1"] >= 2;
    bounding3: x["c3"] <= 0;
""")


In [100]:
CargueroP6.set["cont"] = list(valor.keys())
CargueroP6.param["valor"] = {c: v for c, v in valor.items()}
CargueroP6.param["peso"] = {c: p for c, p in peso.items()}
CargueroP6.param["n"] = {c: num for c, num in n.items()}
CargueroP6.param["cap"] = 400

In [101]:
CargueroP6.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP6.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 187.5
0 simplex iterations


In [102]:
xP6 = CargueroP6.get_variable("x") # Número de cargueros de cada tipo
df_xP6 = xP6.get_values().to_pandas()
print(df_xP6)
valor_total_P6 = CargueroP6.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P6:.3f}")




    x.val
c1   2.45
c2   1.00
c3   0.00
Valor total: 187.500


Actualizamos la cota superior a 187.5.

Ramificamos sobre $P_6$ y resolvemos **$P_7 \ (x_{c_1} \geq 3$)** y **$P_8 \ (x_{c_1} \leq 2$)**

In [103]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP7 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [104]:
CargueroP7.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 1;
    bounding2: x["c1"] >= 3;
    bounding3: x["c3"] <= 0;
""")


In [105]:
CargueroP7.set["cont"] = list(valor.keys())
CargueroP7.param["valor"] = {c: v for c, v in valor.items()}
CargueroP7.param["peso"] = {c: p for c, p in peso.items()}
CargueroP7.param["n"] = {c: num for c, num in n.items()}
CargueroP7.param["cap"] = 400

In [107]:
CargueroP7.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: infeasible problem
0 simplex iterations

suffix dunbdd OUT;


$P_7$ es infactible y agota esta vía.

In [108]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP8 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [109]:
CargueroP8.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 1;
    bounding2: x["c1"] <= 2;
    bounding3: x["c3"] <= 0;
""")


In [110]:
CargueroP8.set["cont"] = list(valor.keys())
CargueroP8.param["valor"] = {c: v for c, v in valor.items()}
CargueroP8.param["peso"] = {c: p for c, p in peso.items()}
CargueroP8.param["n"] = {c: num for c, num in n.items()}
CargueroP8.param["cap"] = 400

In [111]:
CargueroP8.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 183.8709677
0 simplex iterations


In [112]:
xP8 = CargueroP8.get_variable("x") # Número de cargueros de cada tipo
df_xP8 = xP8.get_values().to_pandas()
print(df_xP8)
valor_total_P8 = CargueroP8.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P8:.3f}")




       x.val
c1  2.000000
c2  1.290323
c3  0.000000
Valor total: 183.871


Actualizamos la cota superior a 184 (procedente de $P_4$, que sigue abierto).

Busqueda en profundidad: Ramificamos sobre $P_8$ y resolvemos **$P_9 \ (x_{c_2} \geq 2$)** y **$P_{10} \ (x_{c_2} \leq 1$)**

In [114]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP9 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [115]:
CargueroP9.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 2;
    bounding2: x["c1"] <= 2;
    bounding3: x["c3"] <= 0;
""")


In [116]:
CargueroP9.set["cont"] = list(valor.keys())
CargueroP9.param["valor"] = {c: v for c, v in valor.items()}
CargueroP9.param["peso"] = {c: p for c, p in peso.items()}
CargueroP9.param["n"] = {c: num for c, num in n.items()}
CargueroP9.param["cap"] = 400

In [117]:
CargueroP9.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 175
0 simplex iterations


In [118]:
xP9 = CargueroP9.get_variable("x") # Número de cargueros de cada tipo
df_xP9 = xP9.get_values().to_pandas()
print(df_xP9)
valor_total_P9 = CargueroP9.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P9:.3f}")




    x.val
c1    0.9
c2    2.0
c3    0.0
Valor total: 175.000


Esta rama se poda pues la cota inferior actual del valor objetivo óptimo del problema original es 180 (correspondiente a la solución de $P_2$).

In [119]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP10 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [120]:
CargueroP10.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] =  1;
    bounding2: x["c1"] <= 2;
    bounding3: x["c3"] <= 0;
""")


In [121]:
CargueroP10.set["cont"] = list(valor.keys())
CargueroP10.param["valor"] = {c: v for c, v in valor.items()}
CargueroP10.param["peso"] = {c: p for c, p in peso.items()}
CargueroP10.param["n"] = {c: num for c, num in n.items()}
CargueroP10.param["cap"] = 400

In [122]:
CargueroP10.solve(solver="cplex") # Resolvemos con el solver "cplex"


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 165
0 simplex iterations


In [123]:
xP10 = CargueroP10.get_variable("x") # Número de cargueros de cada tipo
df_xP10 = xP10.get_values().to_pandas()
print(df_xP10)
valor_total_P10 = CargueroP10.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P10:.3f}")




    x.val
c1      2
c2      1
c3      0
Valor total: 165.000


Esta rama termina en una solución entera factible con peor objetivo que la que ya teníamos de la resolución de $P_2$.

Ramificamos sobre $P_4$ y resolvemos **$P_{11} \ (x_{c_2} \geq 2$)** y **$P_{12} \ (x_{c_2} \leq 1$)**

In [124]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP11 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [125]:
CargueroP11.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] >= 2;
    bounding2: x["c1"] <= 1;
""")


In [126]:
CargueroP11.set["cont"] = list(valor.keys())
CargueroP11.param["valor"] = {c: v for c, v in valor.items()}
CargueroP11.param["peso"] = {c: p for c, p in peso.items()}
CargueroP11.param["n"] = {c: num for c, num in n.items()}
CargueroP11.param["cap"] = 400

In [127]:
CargueroP11.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP4.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 180
0 simplex iterations


In [128]:
xP11 = CargueroP11.get_variable("x") # Número de cargueros de cada tipo
df_xP11 = xP11.get_values().to_pandas()
print(df_xP11)
valor_total_P11 = CargueroP11.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P11:.3f}")




    x.val
c1    0.4
c2    2.0
c3    1.0
Valor total: 180.000


Esta rama solo puede dar lugar a una solución entera con valor objetivo igual o peor a la que ya tenemos, así que la podamos (a no ser que queramos explorar la posibilidad de que existan múltiples soluciones, lo cual no haremos aquí).

In [129]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

CargueroP12 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [130]:
CargueroP12.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set cont;                       # Conjunto de contenedores
param valor {cont} >= 0;        # Valor de la carga en cada contenedor
param peso {cont} >= 0;         # Peso de la carga en cada contenedor
param n {cont} >= 0;            # Número de cargamentos de valor y peso dados
param cap >=0;                  # Capacidad del carguero

var x {i in cont}  <= n[i], >=0;  # Número de cargamentos de valor y peso
                                         # dados a fletar en el carguero

maximize valor_total:
    sum{i in cont} valor[i]*x[i];
s.t. capacidad:
    sum{i in cont} peso[i]*x[i] <= cap;
    bounding1: x["c2"] = 1;
    bounding2: x["c1"] <= 1;
""")


In [131]:
CargueroP12.set["cont"] = list(valor.keys())
CargueroP12.param["valor"] = {c: v for c, v in valor.items()}
CargueroP12.param["peso"] = {c: p for c, p in peso.items()}
CargueroP12.param["n"] = {c: num for c, num in n.items()}
CargueroP12.param["cap"] = 400

In [132]:
CargueroP12.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert CargueroP4.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 145
0 simplex iterations


In [133]:
xP12 = CargueroP12.get_variable("x") # Número de cargueros de cada tipo
df_xP12 = xP12.get_values().to_pandas()
print(df_xP12)
valor_total_P12 = CargueroP12.get_objective("valor_total").value()
print(f"Valor total: {valor_total_P12:.3f}")




    x.val
c1      1
c2      1
c3      1
Valor total: 145.000


Obtenemos una solución entera factible con valor objetivo < 180. Ya no quedan pues ningún nodo abierto, por lo que la solución óptima de esta versión reducida del problema del carguero es la proporcionada por el problema $P_2$, esto es, $(3,0,1)$.

Comprobamos que hemos obtenido la solución correcta resolviendo el problema entero original con un solver.

In [60]:
Carguero.set["cont"] = list(valor.keys())
Carguero.param["valor"] = {c: v for c, v in valor.items()}
Carguero.param["peso"] = {c: p for c, p in peso.items()}
Carguero.param["n"] = {c: num for c, num in n.items()}
Carguero.param["cap"] = 400

In [61]:
Carguero.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Carguero.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 180
3 simplex iterations


In [62]:
x = Carguero.get_variable("x") # Número de cargueros de cada tipo
df_x = x.get_values().to_pandas()
print(df_x)
valor_total = Carguero.get_objective("valor_total").value()
print(f"Valor total: {valor_total:.3f}")




    x.val
c1      3
c2      0
c3      1
Valor total: 180.000
